# Random Walks en Multiplex de Aerolíneas

In [1]:
using StatsBase
using DelimitedFiles
using NamedArrays
using CSV
using LightGraphs
using Plots
using DataFrames

In [2]:
cd("$(homedir())/Documents/UNAM/Tesis_Lic")

In [7]:
"""
n_in_degree(M, n)

Función que obtiene el grado de entrada de un cierto nodo n de la red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}
* n nodo a evaluar de tipo Int64

Resultado:
Grado de entrada del nodo n representado por un valor numérico de tipo Int64

## Implementación

### Ejemplos
"""

function n_in_degree(M::Array{Int64,2}, n::Int64)
    grado_entrada = sum(M[n, :])
end

n_in_degree (generic function with 1 method)

In [8]:
"""
in_degree(M)

Función que obtiene el grado de entrada de una red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}

Resultado:
Devuelve un arreglo de tipo Array{Int64,2} que contiene los grados de entrada de todos los nodos n de la red.

## Implementación

### Ejemplos
"""

function in_degree(M::Array)
    grados_entrada = sum(M, dims = 2)
end

in_degree (generic function with 1 method)

In [9]:
"""
n_out_degree(M, n)

Función que obtiene el grado de salida de un cierto nodo n de la red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}
* n nodo a evaluar de tipo Int64

Resultado:
Grado de salida del nodo n representado por un valor numérico de tipo Int64

## Implementación

### Ejemplos
"""

function n_out_degree(M::Array{Int64,2}, n::Int64)
    grado_salida = sum(M[:, n])
end

n_out_degree (generic function with 1 method)

In [10]:
"""
out_degree(M)

Función que obtiene el grado de salida de una red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}

Resultado:
Devuelve un arreglo de tipo Array{Int64,2} que contiene los grados de salida de todos los nodos n de la red.

## Implementación

### Ejemplos
"""

function out_degree(M::Array)
    grados_salida = sum(M, dims = 1)
end

out_degree (generic function with 1 method)

In [28]:
"""
La función r_walks tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
f, nodo inicial de tipo Int64
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz, el cual tiene igual probabilidad de avanzar 
hacia cualquiera de sus vecinos.
Regresa un arreglo que contiene todas las trayectorias que tomó el caminante aleatorio.
"""

function r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)
    nodos = size(M, 1) #aeropuertos  
    paths = zeros(Int64, (iter, steps + 1))
    s = f

    for i in 1:iter
        paths[i, 1] = f
        for p in 1:steps 
            row = M[s, :] #Renglón correspondiente al nodo s
            vecinos = findall(x -> x != 0, row)
            if vecinos == []
                break
            else
                a = sample(vecinos)
                paths[i, p + 1] = a
            s = a
            end
        end
        s = f
    end
    return paths 
end

r_walks (generic function with 1 method)

In [11]:
"""
La función random_walk tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz con igual probabilidad de avanzar 
hacia cualquiera de sus vecinos. Esta función recorre todos los nodos de la red como
nodos iniciales y sobre ellos itera para obtener el número de caminos que se le pidió.
Regresa un arreglo que contiene arreglos asociados a todas las trayectorias que tomó el 
caminante aleatorio para cada nodo inicial de la red.
"""

function random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64) #Debo agregarle el nodo inicial al inicio del path
    random_walks = []
    nodos = size(M, 1)
    for f in 1:nodos #aeropuertos
        paths = r_walks(M, f, steps, iter)
        push!(random_walks, paths)
    end
    random_walks
end

random_walk (generic function with 1 method)

In [12]:
function walk_length(W)
    
    longitudes = Array[]
    c = 0
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos
    longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    for i in 1:v
        #@show i
        for j in 1:s
            #@show j
            for k in 1:t
                #@show k
                
                if W[i][j, :][k] != 0 #W[i][j, :] #renglón a analizar
                    c += 1 
                else
                    break
                    
                    #@show c
                    
                    #@show long #ant[i][j] = W[i][j, :][k - 1]
                    
                end
                
            end
            longz[j] = c - 1
            #@show longz[j]
            c = 0
            
        end
        push!(longitudes, longz)
        #@show longitudes
        longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    end
    return longitudes
end

walk_length (generic function with 1 method)

In [13]:
"""
La función walk_av_length tiene como parámetro W, que es un arreglo de arreglos (que se puede obtener de la 
función walk_length). 
Regresa un arreglo con entradas correspondientes al promedio de las longitudes de 
las caminatas aleatorias para cada nodo de inicio antes de caer en un nodo con grado de salida 0.
"""

function walk_av_length(W) #Llamar a función
    
    proms = []
    wlength = walk_length(W)
    
    for i in 1:length(wlength)
        av = mean(wlength[i])
        push!(proms, av)
    end
    proms
end

walk_av_length (generic function with 1 method)

In [14]:
"""
La función n_cobertura tiene como parámetro W que es un arreglo de arreglos.
Regresa los nodos diferentes a los que llega el caminante.
"""

function n_cobertura(W)
    
    cobert = []
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos
    #visit = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    visit = []
    
    for i in 1:v        
        for j in 1:s
            u = unique(W[i][j, :])
            push!(visit, u)
        end
        push!(cobert, visit)
        visit = []
    end
    return cobert
end

n_cobertura (generic function with 1 method)

In [15]:
"""
La función cobertura tiene como parámetro W que es un arreglo de arreglos.
Regresa el número de nodos diferentes a los que llega el caminante.
"""

#Tal vez falta llamar dentro de la función a la función n_cobertura

function cobertura(W) 
    
    cobert = []
    
    v = length(W) #tamaño de la matriz, número de nodos
    x = n_cobertura(W)
    s = size(x[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    for i in 1:v        
        for j in 1:s
            l = length(x[i][j])
            longz[j] = l
        end
        push!(cobert, longz)
        longz = zeros(Int64, s)
    end
    return cobert
end

cobertura (generic function with 1 method)

In [16]:
"""

"""

#Tal vez falta llamar dentro de la función a la función n_cobertura y cobertura

function n_length_av(W) #Esta es la importante
    
    longitud = []
    x = cobertura(W)
    
    for i in 1:length(x)
        l = mean(x[i])
        push!(longitud, l)
    end
    longitud
end

n_length_av (generic function with 1 method)

In [17]:
"""

"""

#Tal vez falta llamar dentro de la función a la función n_cobertura y cobertura y n_length_av

function length_av(W)
    x = n_length_av(W)
    promedio = mean(x)
    return promedio

end

length_av (generic function with 1 method)

In [18]:
"""

"""


function max_length(W)
    max = []
    x = cobertura(W)
    for i in 1:length(x)
        m = maximum(x[i])
        push!(max, m)
    end
    max
end

max_length (generic function with 1 method)

In [19]:
"""

"""

#Tal vez falta llamar dentro de la función a la función n_cobertura y cobertura

function Max_length(W)
    x = max_length(W)
    m = maximum(x)
    return m
end

Max_length (generic function with 1 method)

In [20]:
"""

"""

#Tal vez falta llamar dentro de la función a la función n_cobertura y cobertura

function av_max_length(W)
    x = max_length(W)
    prom = mean(x)
    return prom
end

av_max_length (generic function with 1 method)

In [21]:
"""

"""

#Tal vez falta llamar dentro de la función a la función n_cobertura y cobertura

function min_length(W)
    min = []
    x = cobertura(W)
    for i in 1:length(x)
        m = minimum(x[i])
        push!(min, m)
    end
    min
end

min_length (generic function with 1 method)

In [22]:
"""

"""

#Tal vez falta llamar dentro de la función a la función n_cobertura y cobertura

function Min_length(W)
    x = min_length(W)
    m = minimum(x)
    return m
end

Min_length (generic function with 1 method)

In [23]:
"""

"""

#Tal vez falta llamar dentro de la función a la función n_cobertura y cobertura

function av_min_length(W)
    x = min_length(W)
    prom = mean(x)
    return prom
end

av_min_length (generic function with 1 method)

In [ ]:
function atractor(W) #Debo llamar a walks
   atractores = []
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos
    #visit = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    visit = []
    
    for i in 1:v        
        for j in 1:s
            if 
            a = W[i][j, :])
            push!(visit, u)
        end
        push!(cobert, visit)
        visit = []
    end
    return cobert
end

## Datos

In [3]:
m17 = readdlm("data/multi_17_du.csv", ',')

3190×3190 Array{Any,2}:
 ""      "AER"   "ASF"   "CEK"   "DME"  …   "BSS"   "AEX"   "GCK"   "MGM"
 "AER"  0       0       0       1          0       0       0       0     
 "ASF"  0       0       0       1          0       0       0       0     
 "CEK"  0       0       0       1          0       0       0       0     
 "DME"  1       1       1       0          0       0       0       0     
 "EGO"  0       0       0       1       …  0       0       0       0     
 "GYD"  0       0       0       1          0       0       0       0     
 "KGD"  0       0       0       1          0       0       0       0     
 "KZN"  1       1       1       1          0       0       0       0     
 "LED"  1       1       1       1          0       0       0       0     
 "MRV"  0       1       0       1       …  0       0       0       0     
 "NBC"  0       0       0       1          0       0       0       0     
 "NJC"  0       0       0       1          0       0       0       0     
 ⋮            

In [4]:
multi17 = m17[2:end, :];
multi17 = multi17[:, 2:end];

In [5]:
nombres_aeropuertos = m17[1,:]

3190-element Array{Any,1}:
 ""   
 "AER"
 "ASF"
 "CEK"
 "DME"
 "EGO"
 "GYD"
 "KGD"
 "KZN"
 "LED"
 "MRV"
 "NBC"
 "NJC"
 ⋮    
 "LPS"
 "ORX"
 "BVS"
 "MTE"
 "DLZ"
 "UII"
 "ZBF"
 "CMP"
 "BSS"
 "AEX"
 "GCK"
 "MGM"

In [26]:
multiplex = convert(Array{Int64,2}, multi17)

3189×3189 Array{Int64,2}:
 0  0  0  1  0  0  0  1  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  1  1  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  1  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  0  1  1  1  1  1  1  1  1  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  1  1  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  1  1  1  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  1  0  0  0  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  0  0  0  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  1  1  1  0  1  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  0  1  0  0  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  1  0  0  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0

In [29]:
caminata_M = @time random_walk(multiplex, 100, 5)

 73.894156 seconds (19.17 M allocations: 38.698 GiB, 5.20% gc time)


3189-element Array{Any,1}:
 [1 671 … 1193 1129; 1 1531 … 188 52; … ; 1 837 … 161 1925; 1 4 … 1869 2332] 
 [2 8 … 451 1024; 2 1829 … 1032 135; … ; 2 4 … 60 1761; 2 232 … 0 0]         
 [3 90 … 1710 1600; 3 4 … 1656 1930; … ; 3 478 … 307 1548; 3 90 … 1979 1980] 
 [4 730 … 1870 2386; 4 11 … 1794 886; … ; 4 764 … 938 124; 4 53 … 1078 71]   
 [5 753 … 1158 1175; 5 8 … 580 307; … ; 5 759 … 1015 170; 5 8 … 1391 1081]   
 [6 2282 … 55 1233; 6 451 … 715 217; … ; 6 727 … 1868 358; 6 1561 … 260 336] 
 [7 594 … 398 2930; 7 5 … 726 682; … ; 7 9 … 405 398; 7 1527 … 1385 429]     
 [8 423 … 164 1009; 8 594 … 684 1851; … ; 8 9 … 2385 2829; 8 15 … 357 361]   
 [9 473 … 0 0; 9 2231 … 661 449; … ; 9 220 … 252 1085; 9 724 … 594 429]      
 [10 232 … 715 1546; 10 6 … 438 1226; … ; 10 6 … 428 231; 10 594 … 358 759]  
 [11 4 … 220 1218; 11 724 … 289 264; … ; 11 727 … 1149 1150; 11 727 … 1752 4]
 [12 4 … 1015 2035; 12 594 … 163 560; … ; 12 349 … 338 1110; 12 4 … 305 1012]
 [13 2275 … 1024 168; 13 671 … 1448 1

In [48]:
@time walk_length(caminata_M);

  1.840936 seconds (5.50 M allocations: 1.365 GiB, 6.08% gc time)


In [31]:
@time walk_av_length(caminata_M)

  1.863774 seconds (5.68 M allocations: 1.374 GiB, 6.06% gc time)


3189-element Array{Any,1}:
 100.0
  96.6
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
  85.8
 100.0
 100.0
 100.0
 100.0
   ⋮  
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [47]:
@time length_av(caminata_M)

  0.130367 seconds (353.52 k allocations: 104.501 MiB, 19.28% gc time)


74.47895892129192

In [51]:
@time n_cobertura(caminata_M)

  0.132426 seconds (321.61 k allocations: 103.256 MiB, 21.81% gc time)


3189-element Array{Any,1}:
 Any[[1, 671, 358, 2334, 162, 2336, 1093, 1656, 362, 636  …  3103, 3128, 338, 1170, 1115, 1200, 894, 1184, 1201, 1193], [1, 1531, 4, 57, 2017, 1988, 481, 661, 220, 1587  …  456, 189, 495, 193, 1016, 44, 31, 716, 467, 188], [1, 1531, 473, 183, 1567, 467, 1342, 441, 1358, 223  …  304, 352, 670, 2276, 20, 594, 725, 729, 2744, 724], [1, 837, 423, 454, 434, 55, 1864, 469, 130, 470  …  1523, 634, 1974, 1015, 627, 2864, 1347, 1650, 1640, 1925], [1, 4, 10, 724, 9, 130, 40, 131, 1488, 2225  …  327, 1687, 259, 1632, 326, 316, 2323, 2331, 1869, 2332]]       
 Any[[2, 8, 1752, 727, 4, 1839, 178, 1430, 1432, 1603  …  264, 1613, 650, 630, 467, 1216, 1049, 1351, 451, 1024], [2, 1829, 1819, 1532, 1832, 723, 14, 26, 9, 423  …  491, 486, 999, 439, 684, 1527, 1091, 435, 135, 1032], [2, 4, 346, 352, 493, 727, 305, 878, 313, 289  …  1130, 886, 1150, 620, 1195, 31, 46, 220, 491, 191], [2, 4, 244, 15, 2797, 762, 349, 2870, 189, 432  …  999, 90, 998, 495, 476, 1048, 227, 1049, 471, 

In [36]:
@time cobertura(caminata_M)

  0.160455 seconds (343.94 k allocations: 104.292 MiB, 32.85% gc time)


3189-element Array{Any,1}:
 [79, 77, 74, 77, 86]
 [75, 83, 80, 87, 70]
 [86, 83, 74, 85, 81]
 [85, 76, 88, 88, 78]
 [84, 84, 83, 83, 78]
 [83, 73, 83, 85, 81]
 [82, 79, 78, 85, 81]
 [83, 79, 83, 82, 87]
 [29, 84, 85, 79, 82]
 [77, 87, 78, 80, 80]
 [87, 83, 66, 89, 80]
 [79, 82, 80, 79, 86]
 [83, 86, 83, 87, 85]
 ⋮                   
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     

In [65]:
@time caminata_M
@time n_length_av(caminata_M)

  0.000001 seconds (3 allocations: 144 bytes)
  0.286188 seconds (347.15 k allocations: 104.404 MiB, 63.52% gc time)


3189-element Array{Any,1}:
 78.6
 79.0
 81.8
 83.0
 82.4
 81.0
 81.0
 82.8
 71.8
 80.4
 81.0
 81.2
 84.8
  ⋮  
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0

In [49]:
@time max_length(caminata_M);

  0.128620 seconds (343.96 k allocations: 104.355 MiB, 16.06% gc time)


In [42]:
@time Max_length(caminata_M)

  0.178791 seconds (377.12 k allocations: 106.120 MiB, 15.99% gc time)


96

In [43]:
@time av_max_length(caminata_M)

  0.135874 seconds (348.09 k allocations: 104.457 MiB, 16.29% gc time)


83.74788334901223

In [50]:
@time min_length(caminata_M);

  0.285365 seconds (343.96 k allocations: 104.355 MiB, 62.80% gc time)


In [45]:
@time Min_length(caminata_M)

  0.172192 seconds (377.21 k allocations: 106.129 MiB, 16.34% gc time)


2

In [46]:
@time av_min_length(caminata_M)

  0.293863 seconds (348.06 k allocations: 104.456 MiB, 60.72% gc time)


59.80620884289746

In [ ]:
@time atractor(caminata_M)